In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
KEY = os.getenv("OPENAI_API_KEY")

In [6]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8520\524943024.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)


In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice 1",
            "b": "choice 2",
            "c": "choice 3",
            "d": "choice 4",
        },
        "correct": "answer"
    },
    "2":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice 1",
            "b": "choice 2",
            "c": "choice 3",
            "d": "choice 4",
        },
        "correct": "answer"
    },
    "3":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice 1",
            "b": "choice 2",
            "c": "choice 3",
            "d": "choice 4",
        },
        "correct": "answer"
    }
}

In [11]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQ's
### RESPONSE_JSON
{response_json}
"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "tone", "subject", "response_json"],
    template=TEMPLATE
) 

In [13]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8520\4109068563.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [15]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
    You need to evaluate the complexity if the questions and give a complete analysis of the quiz. Only use at max words for complexity, 
    if he quiz is not at per with the congnitive and analytical abilities of the students,\
    update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities.
    Quiz_MCQs:
    {quiz}
    
    Check from an expert English writer of the above quiz:
        
"""

In [16]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [17]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [18]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "tone", "subject", "response_json"], 
    output_variables=["quiz","review"], 
    verbose=True
)

In [35]:
file_path = r"D:\langchain\QuizSpare\QuizKraft\data.txt"

In [36]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [22]:
print(TEXT)

Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

AI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications,

In [23]:
#Serializing the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice 1", "b": "choice 2", "c": "choice 3", "d": "choice 4"}, "correct": "answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice 1", "b": "choice 2", "c": "choice 3", "d": "choice 4"}, "correct": "answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice 1", "b": "choice 2", "c": "choice 3", "d": "choice 4"}, "correct": "answer"}}'

In [24]:
NUMBER = 5
SUBJECT = "Artificial Intelligence"
TONE = "Simple"

In [37]:
# Setup token usage tracking in langchain

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

AI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, ma

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [26]:
cb

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0

In [27]:
response

NameError: name 'response' is not defined

In [73]:
quiz = response.get("quiz")

In [74]:
quiz

'{\n"1": {\n"mcq": "What is the broadest sense of artificial intelligence (AI)?",\n"options": {\n"a": "Intelligence exhibited by humans",\n"b": "Intelligence exhibited by machines",\n"c": "Intelligence exhibited by animals",\n"d": "Intelligence exhibited by plants"\n},\n"correct": "b"\n},\n"2": {\n"mcq": "Who was the first person to conduct substantial research in machine intelligence?",\n"options": {\n"a": "Alan Turing",\n"b": "Albert Einstein",\n"c": "Isaac Newton",\n"d": "Charles Babbage"\n},\n"correct": "a"\n},\n"3": {\n"mcq": "Which technology surpassed all previous AI techniques after 2012?",\n"options": {\n"a": "Machine Learning",\n"b": "Deep Learning",\n"c": "Artificial Neural Networks",\n"d": "Statistical Analysis"\n},\n"correct": "b"\n},\n"4": {\n"mcq": "What is one of the long-term goals of AI research?",\n"options": {\n"a": "Creating robots with emotions",\n"b": "Achieving general intelligence",\n"c": "Developing time travel",\n"d": "Building flying cars"\n},\n"correct": "b

In [75]:
quiz = json.loads(quiz)

In [76]:
quiz_table_data = []

for key, value in quiz.items():

    mcq = value["mcq"]
    if "options" in value:
        options = " | ".join(
            [
                f"{option}: {option_value}"
                for option, option_value in value["options"].items()
            ]
        )
    else:
        options = "No options available"
    correct = value.get("correct", "No correct answer specified")
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [77]:
quiz_table_data

[{'MCQ': 'What is the broadest sense of artificial intelligence (AI)?',
  'Choices': 'a: Intelligence exhibited by humans | b: Intelligence exhibited by machines | c: Intelligence exhibited by animals | d: Intelligence exhibited by plants',
  'Correct': 'b'},
 {'MCQ': 'Who was the first person to conduct substantial research in machine intelligence?',
  'Choices': 'a: Alan Turing | b: Albert Einstein | c: Isaac Newton | d: Charles Babbage',
  'Correct': 'a'},
 {'MCQ': 'Which technology surpassed all previous AI techniques after 2012?',
  'Choices': 'a: Machine Learning | b: Deep Learning | c: Artificial Neural Networks | d: Statistical Analysis',
  'Correct': 'b'},
 {'MCQ': 'What is one of the long-term goals of AI research?',
  'Choices': 'a: Creating robots with emotions | b: Achieving general intelligence | c: Developing time travel | d: Building flying cars',
  'Correct': 'b'},
 {'MCQ': 'Which field does AI draw upon for its research?',
  'Choices': 'a: Biology | b: Psychology | c:

In [79]:
quiz_df = pd.DataFrame(quiz_table_data)

In [80]:
quiz_df

,MCQ,Choices,Correct
0,What is the broadest sense of artificial intel...,a: Intelligence exhibited by humans | b: Intel...,b
1,Who was the first person to conduct substantia...,a: Alan Turing | b: Albert Einstein | c: Isaac...,a
2,Which technology surpassed all previous AI tec...,a: Machine Learning | b: Deep Learning | c: Ar...,b
3,What is one of the long-term goals of AI resea...,a: Creating robots with emotions | b: Achievin...,b
4,Which field does AI draw upon for its research?,a: Biology | b: Psychology | c: Geology | d: M...,b


In [81]:
quiz_df.to_csv("aiquiz.csv", index=False)

In [1]:
import datetime

In [12]:
datetime.datetime.now()

datetime.datetime(2024, 5, 19, 5, 47, 2, 490451)